(example.e4cv.custom-class)=
# Example 4-circle diffractometer custom Python class

It's always possible to define your own subclass of
{class}`~hklpy2.diffract.DiffractometerBase()` when you need more control than
provided by {func}`~hklpy2.diffract.creator()`.

Here's a brief example of a 4-circle diffractometer with a custom Python class.
Add many additional axes, both in real (rotation angle) space and in reciprocal
(pseudo) space.

In [ ]:
import hklpy2
from ophyd import Component as Cpt
from ophyd import EpicsMotor
from ophyd import Kind
from ophyd import PseudoSingle
from ophyd import SoftPositioner

NORMAL_HINTED = Kind.hinted | Kind.normal


class Fourc(hklpy2.DiffractometerBase):
    """Test case."""

    # Pseudo-space axes, in order expected by hkl_soleil E4CV, engine="hkl"
    h = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    # Real-space axes, in our own order..
    # Use different names than the solver for some axes
    ttheta = Cpt(EpicsMotor, "m29", kind=NORMAL_HINTED)
    theta = Cpt(EpicsMotor, "m30", kind=NORMAL_HINTED)
    chi = Cpt(EpicsMotor, "m31", kind=NORMAL_HINTED)
    phi = Cpt(EpicsMotor, "m32", kind=NORMAL_HINTED)

    # Pseudo-space extra axes used in a couple modes.
    h2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    # real-space extra axis used in a couple modes
    psi = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

    # another Component, not used (yet)
    energy = Cpt(SoftPositioner, limits=(5, 35), init_pos=12.4, kind=NORMAL_HINTED)

    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            solver="hkl_soleil",
            geometry="E4CV",
            solver_kwargs=dict(engine="hkl"),
            **kwargs,
        )
        # self.core.auto_assign_axes()  # When axes are specified in expected order.
        self.core.assign_axes(
            "h k l".split(),
            # Just the axes expected by hkl_soleil E4CV, in order.
            "theta chi phi ttheta".split(),
        )


fourc = Fourc("gp:", name="fourc")
fourc

Fourc(prefix='gp:', name='fourc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['h', 'h.readback', 'h.setpoint', 'k', 'k.readback', 'k.setpoint', 'l', 'l.readback', 'l.setpoint', 'ttheta', 'ttheta.user_readback', 'ttheta.user_setpoint', 'theta', 'theta.user_readback', 'theta.user_setpoint', 'chi', 'chi.user_readback', 'chi.user_setpoint', 'phi', 'phi.user_readback', 'phi.user_setpoint', 'h2', 'h2.readback', 'h2.setpoint', 'k2', 'k2.readback', 'k2.setpoint', 'l2', 'l2.readback', 'l2.setpoint', 'psi', 'energy'], configuration_attrs=['geometry', 'solver', 'wavelength', 'h', 'k', 'l', 'ttheta', 'ttheta.user_offset', 'ttheta.user_offset_dir', 'ttheta.velocity', 'ttheta.acceleration', 'ttheta.motor_egu', 'theta', 'theta.user_offset', 'theta.user_offset_dir', 'theta.velocity', 'theta.acceleration', 'theta.motor_egu', 'chi', 'chi.user_offset', 'chi.user_offset_dir', 'chi.velocity', 'chi.acceleration', 'chi.motor_egu', 'phi', 'phi.user_offset', 'phi.user_o

In [2]:
print("Brief 'where' report:")
fourc.wh()


Brief 'where' report:


/home/zorinvm/Documents/projects/prjemian/hklpy2/hklpy2/misc.py:316: UserWarning:  Extra inputs will be ignored. Expected 4. Received input=FourcRealPos(ttheta=0.0, theta=0.0, chi=0.0, phi=0.0, psi=0, energy=12.4), names=['ttheta', 'theta', 'chi', 'phi']
  warnings.warn(


h=0, k=0, l=0, h2=0, k2=0, l2=0
wavelength=1.0
ttheta=0, theta=0, chi=0, phi=0, psi=0, energy=12.4


In [3]:
print("Full 'where' report:")
fourc.wh(full=True)

Full 'where' report:
diffractometer='fourc'
HklSolver(name='hkl_soleil', version='5.1.2', geometry='E4CV', engine_name='hkl', mode='bissector')
Sample(name='sample', lattice=Lattice(a=1, system='cubic'))
Orienting reflections: []
U=[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
UB=[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
constraint: -180.0 <= ttheta <= 180.0
constraint: -180.0 <= theta <= 180.0
constraint: -180.0 <= chi <= 180.0
constraint: -180.0 <= phi <= 180.0
constraint: -180.0 <= psi <= 180.0
constraint: -180.0 <= energy <= 180.0
h=0, k=0, l=0, h2=0, k2=0, l2=0
wavelength=1.0
ttheta=0, theta=0, chi=0, phi=0, psi=0, energy=12.4


Use the lower level methods to compute `forward()` and `inverse()` transformations.

In [4]:
fourc.forward(1, 0, 0)  # Shows the default choice.

/home/zorinvm/Documents/projects/prjemian/hklpy2/hklpy2/misc.py:316: UserWarning:  Extra inputs will be ignored. Expected 3. Received input=FourcPseudoPos(h=1, k=0, l=0, h2=0, k2=0, l2=0), names=['h', 'k', 'l']
  warnings.warn(


FourcRealPos(ttheta=30.00000000079, theta=-4.747e-09, chi=90.00000000447, phi=60.00000000158, psi=0, energy=12.4)

In [5]:
fourc.core.forward(dict(h=1, k=0, l=0))  # Shows ALL the possibilities.

[FourcRealPos(ttheta=-30.000000181209, theta=-9.5582e-08, chi=-89.999999974224, phi=-60.000000362418, psi=0, energy=12.4),
 FourcRealPos(ttheta=30.000000181209, theta=-9.5582e-08, chi=90.000000025776, phi=60.000000362418, psi=0, energy=12.4),
 FourcRealPos(ttheta=-30.000000181209, theta=-9.5582e-08, chi=150.000000388193, phi=60.000000396423, psi=0, energy=12.4),
 FourcRealPos(ttheta=-149.999999818791, theta=-9.5582e-08, chi=29.999999663358, phi=-60.000000465387, psi=0, energy=12.4),
 FourcRealPos(ttheta=30.000000181209, theta=-9.5582e-08, chi=-150.000000336642, phi=-60.000000465387, psi=0, energy=12.4),
 FourcRealPos(ttheta=-149.999999818791, theta=-9.5582e-08, chi=-89.999999974224, phi=60.000000362418, psi=0, energy=12.4),
 FourcRealPos(ttheta=-30.000000181209, theta=-179.999999904418, chi=90.000000025776, phi=-60.000000362418, psi=0, energy=12.4),
 FourcRealPos(ttheta=30.000000181209, theta=-179.999999904418, chi=-89.999999974224, phi=60.000000362418, psi=0, energy=12.4),
 FourcRealP

In [6]:
fourc.core.inverse(dict(ttheta=30, theta=15, chi=0, phi=90))

{'h': -0.366025403784,
 'k': 0.353553390593,
 'l': 1.319479216882,
 'h2': 0,
 'k2': 0,
 'l2': 0}